# Waterhackweek 2020 Project - SnowGW: basin water budget components

Based off of original code by [Emilio Mayorga](https://github.com/emiliom), University of Washington &mdash; [WaterHackWeek 2020](https://waterhackweek.github.io/). See Emilio's original script [mashup_waterbudget.ipynb](https://github.com/waterhackweek/waterdata/blob/master/mashup_waterbudget.ipynb) for helpful information on data sources, packages, and functions.

## Goal

* Construct time series visualizations at daily and/or monthly timescales that show water budget components for basins with snow-groundwater dynamics of interest. (Interim)

## Objective

**Construct and analyze (sub)watershed-scale water budget components for the two example watersheds using geospatial watershed layers and a wide variety of water data, including:**

* HUC8 basin boundaries (and associated HUC10 and HUC12 sub-basins)
* Daymet precipitation and SWE (gridded, daily)
* MODIS SSEBop evapotranspiration (gridded, monthly)
* GRACE water storage anomalies (gridded, monthly)
* USGS NWIS discharge (point, daily)
* USGS groundwater level measurements (point, daily)
* SNOTEL SWE (point, daily)

## Cases

### ONLY works for OR case right now!
                                                  
**Clackamas, OR**
* HUC-8 Subbasin ID 17090011
* size 2442 km^2
* slope 24.2%
* Total stream length (km) 1810.4 km
* Land cover: evergreen forest (73%), shrub/scrub (10%)

**Upper South Platte, CO**
* HUC-8 Subbasin ID 10190002 
* size: 4,797 km²
* slope: 22.3%
* Total stream length (km) 2812.4 km
* Land cover: evergreen forest (50%), shrub/ scrub: 18%, grassland: 15%

----------------

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">Setup</b></br>
</div>

## Import packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
import contextily as cx
import folium

import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import xarray as xr
import rioxarray
import regionmask as rm

from shapely.geometry import box
import pynhd as nhd
import pygeoogc as geoogc
import pygeoutils as geoutils
import hydrodata as hd

## Set the time period

To be reused with every dataset.

In [ ]:
date_start = '2009-01-01'
date_end = '2019-01-01'

## Select basin

Select one of the case study basins' HUC8 codes (OR: `17090011`; CO: `10190002`)

We'll use the shorthand "sb" to refer to the "selected basin".

In [ ]:
#sb_huc08code = '10190002'

sb_huc08code = '17090011'

----------------

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">Format basin</b></br>
</div>

### Create HUC10 subwatersheds by first reading HUC12 subwatersheds

Request all HUC12 watersheds whose HUC12 code includes the sb code.

In [ ]:
def wfs_getfeatures_cqlfilter(wd, cql_filter=None):
    """
    Use hydrodata packages to issue and process a OpenGeospatial Consortium (OGC) Web Feature Service (WFS) 
    request for WBD watersheds, with an optional filter to obtain only the watersheds we want.
    Returns a nice and clean GeoPandas GeoDataframe in "lat-lon" projection (epsg:4326)
    """
    payload = {
        "service": "wfs",
        "request": "GetFeature",
        "version": wd.version,
        "outputFormat": wd.outformat,
        "typeName": wd.layer,
    }
    if type(cql_filter) is str:
        payload["cql_filter"] = cql_filter

    r = geoogc.RetrySession().get(wd.url, payload)
    
    return geoutils.json2geodf(r.json(), "epsg:4326", crs="epsg:4326")

In [ ]:
sb_wdhuc12 = nhd.WaterData('huc12', crs='epsg:4269')

In [ ]:
print(sb_wdhuc12.get_validnames())

In [ ]:
# HUC filter (cql_filter) will be the string "huc12 LIKE '17030001%'"
sb_huc12_gdf = wfs_getfeatures_cqlfilter(
    sb_wdhuc12, 
    cql_filter=f"huc12 LIKE '{sb_huc08code}%'"
)

sb_huc12_gdf.head(2)

Create `huc10` column from `huc12` codes.

In [ ]:
sb_huc12_gdf['huc10'] = sb_huc12_gdf.huc12.str[:-2]
cols = ['huc10', 'areasqkm', 'geometry']
sb_huc10_gdf = sb_huc12_gdf[cols].dissolve(by='huc10', aggfunc='sum', as_index=False)

len(sb_huc10_gdf)

In [ ]:
sb_huc10_gdf.head(2)

Use the `total_bounds` GeoDataFrame function to extract the outer bounding box coordinates, then create a simple GeoSeries out of that rectangle for reuse later on.

In [ ]:
sb_huc10_gdf.total_bounds

In [ ]:
sb_bbox = gpd.GeoSeries(box(*sb_huc10_gdf.total_bounds), crs=sb_huc10_gdf.crs)

Set `xarray` spatial (lat & lon) "slices" here, for reuse below when doing lat-lon region clipping (via the `.sel` method) of `xarray` datasets.

In [ ]:
sb_lon_slice = slice(sb_bbox.bounds.minx[0], sb_bbox.bounds.maxx[0])
sb_lat_slice = slice(sb_bbox.bounds.miny[0], sb_bbox.bounds.maxy[0])

Plot HUC8 basin + HUC10 sub-basins

In [ ]:
# Note: There are other basemap tiles besides StamenTerrain
m = folium.Map(tiles='StamenTerrain', attr='ESRI')

In [ ]:
# Bounding box
folium.GeoJson(
    sb_bbox.geometry,
    style_function=lambda feature: {'color': 'red', 'fillOpacity': 0.0}
).add_to(m)

# Set the map extent (bounds) to the extent of the bounding box
m.fit_bounds(m.get_bounds())

# HUC10 sub-watershed polygons
for i, i_gdf in sb_huc10_gdf.iterrows():
    folium.GeoJson(
        i_gdf.geometry,
        style_function=lambda feature: {'color': 'black', 'fillOpacity': 0.0},
        tooltip=i_gdf['huc10']
    ).add_child(
        folium.Popup(f"<b>HUC10:</b><br> {i_gdf['huc10']}<br><b>watershed area:</b><br> {i_gdf['areasqkm']:.0f} km^2")
    ).add_to(m)

In [ ]:
m

----------------

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">Read data, and extract over selection (time and basin) </b></br>
</div>

## Precipitation and SWE from DayMet

### Get metadata ("lazy loading")

In [ ]:
daymet_opendap_url = "https://thredds.daac.ornl.gov/thredds-daymet/dodsC/daymet-v3-agg/na.ncml"
    
daymet_ds = xr.open_dataset(daymet_opendap_url)

### Get the selected basin bounding box in the DayMet projection coordinates

In [ ]:
daymet_ds_grid_mapping_name = 'lambert_conformal_conic'

In [ ]:
# DayMet crs information is in: ds.lambert_conformal_conic ('lcc')
lcc_crs = CRS.from_cf(daymet_ds[daymet_ds_grid_mapping_name].attrs)

In [ ]:
# Reproject our bounding box polygon into this new coordinate system, to use in our request
sb_bbox_lcc = sb_bbox.to_crs(lcc_crs)

In [ ]:
sb_lcc_bnd = sb_bbox_lcc.bounds

### Get the DayMet data for the selected XYT "cube"

In [ ]:
sb_daymet_da = daymet_ds.sel(
    time=slice(date_start, date_end),
    x=slice(sb_lcc_bnd.minx[0], sb_lcc_bnd.maxx[0]),
    y=slice(sb_lcc_bnd.maxy[0], sb_lcc_bnd.miny[0]) # note the reversed y values!
)

Select precipitation and SWE

In [ ]:
sb_daymet_da_prcp = sb_daymet_da.prcp
sb_daymet_da_swe = sb_daymet_da.swe

### Reproject to lat-lon (to use with other data)

In [ ]:
def reproject_da_as_ds(da, from_grid_mapping_name, from_crs, to_crs):
    """
    Reproject an xarray DataArray using rioxarray and the expectation of CF convention 
    for projection information.
    This function is fairly generic and can be reused with other datasets with only small changes.
    Returns an xarray DataSet with the expected, CF-convention projection information.
    """
    ds = da.to_dataset()
    # Use rioxarray to improve (clean up) the CRS metadata, 
    # so it can be used in the reprojection step
    ds.rio.write_crs(from_crs.to_string(), inplace=True)
    
    # Reproject the xarray dataset
    # Note: the drop_vars step could be done outside this function, since it's source-data cleanup
    reproj_ds = (
        ds
        .drop_vars(['lon', 'lat'])
        .rio.reproject(to_crs)
    )
    
    # Clean up the projection information and coordinates of the reprojected dataset
    to_grid_mapping_name = reproj_ds[from_grid_mapping_name].attrs['grid_mapping_name']
    reproj_ds = (
        reproj_ds
        .rio.set_spatial_dims('x', 'y')
        .rio.write_crs(to_crs)
        .rename({"x": "lon", "y": "lat", from_grid_mapping_name: to_grid_mapping_name}) 
        # this last step applies only when reprojecting to lat-lon
    )
    reproj_ds.attrs['grid_mapping'] = to_grid_mapping_name
    reproj_ds[da.name].attrs['grid_mapping'] = to_grid_mapping_name
    
    return reproj_ds

In [ ]:
sb_daymet_ds_prcp = reproject_da_as_ds(sb_daymet_da_prcp, daymet_ds_grid_mapping_name, lcc_crs, "epsg:4326")
sb_daymet_ds_swe = reproject_da_as_ds(sb_daymet_da_swe, daymet_ds_grid_mapping_name, lcc_crs, "epsg:4326")

Quick check on the resulting cell sizes, in degrees

In [ ]:
def print_cellsizes(dimcoord):
    print(dimcoord.values[1] - dimcoord.values[0], dimcoord.values[-1] - dimcoord.values[-2])

In [ ]:
print_cellsizes(sb_daymet_ds_prcp.lon)
print_cellsizes(sb_daymet_ds_swe.lon)

In [ ]:
print_cellsizes(sb_daymet_ds_prcp.lat)
print_cellsizes(sb_daymet_ds_swe.lat)

Clip to basin polygon boundary

In [ ]:
sb_daymet_ds_prcp = sb_daymet_ds_prcp.rio.clip(sb_huc10_gdf.geometry, all_touched=True)
sb_daymet_ds_swe = sb_daymet_ds_swe.rio.clip(sb_huc10_gdf.geometry, all_touched=True)

Look ("5th time step from the end")

In [ ]:
f, ax = plt.subplots(1, figsize=(7, 7))
sb_daymet_ds_prcp.prcp.isel(time=-5).plot(ax=ax)
sb_huc10_gdf.plot(ax=ax, edgecolor='white', facecolor='none');

In [ ]:
f, ax = plt.subplots(1, figsize=(7, 7))
sb_daymet_ds_swe.swe.isel(time=-5).plot(ax=ax)
sb_huc10_gdf.plot(ax=ax, edgecolor='white', facecolor='none');

## ETa from MODIS SSEBop

In [ ]:
ssebop_opendap_url = "https://cida.usgs.gov/thredds/dodsC/ssebopeta/monthly"

ssebop_ds = xr.open_dataset(ssebop_opendap_url)

In [ ]:
ssebop_ds

In [ ]:
ssebop_ds.rio.write_crs(CRS.from_cf(ssebop_ds['crs'].attrs), inplace=True);

In [ ]:
ssebop_ds.time.values[0:3]

In [ ]:
sb_ssebop_et_ds = ssebop_ds.sel(time=slice(date_start, date_end), lon=sb_lon_slice, lat=sb_lat_slice)

sb_ssebop_et_ds

In [ ]:
sb_ssebop_et_ds = sb_ssebop_et_ds.rio.clip(sb_huc10_gdf.geometry, all_touched=True)

In [ ]:
f, ax = plt.subplots(1, figsize=(7, 7))
sb_ssebop_et_ds.et.isel(time=6).plot(ax=ax)
sb_huc10_gdf.plot(ax=ax, edgecolor='white', facecolor='none');

## TWS anomalies from GRACE

In [ ]:
grace_opendap_url = "https://podaac-opendap.jpl.nasa.gov/opendap/allData/tellus/L3/mascon/RL06/JPL/v02/CRI/netcdf/GRCTellus.JPL.200204_202007.GLO.RL06M.MSCNv02CRI.nc"

grace_ds = xr.open_dataset(grace_opendap_url)

In [ ]:
grace_ds

In [ ]:
grace_ds.lon.values.min(), grace_ds.lon.values.max()

**NOTE:** `lon` values are 0 - 360 (starting from the anti-meridian, "-180") rather than -180 to 180 as the previous datasets; add an offset in the lon slice specification, and shift the xarray dataset to make it work with the region mask and GeoDataFrame overlays.

In [ ]:
sb_grace_lwe_thickness_ds = grace_ds.sel(
    time=slice(date_start, date_end), 
    lon=slice(360 + sb_bbox.bounds.minx[0] - 0.5, 360 + sb_bbox.bounds.maxx[0] + 0.5), 
    lat=slice(sb_bbox.bounds.miny[0] - 0.5, sb_bbox.bounds.maxy[0] + 0.5), 
)

sb_grace_lwe_thickness_ds

In [ ]:
sb_grace_lwe_thickness_ds.coords['lon'] = (sb_grace_lwe_thickness_ds.coords['lon'] + 180) % 360 - 180
sb_grace_lwe_thickness_ds = sb_grace_lwe_thickness_ds.sortby(sb_grace_lwe_thickness_ds.lon)

In [ ]:
sb_grace_lwe_thickness_ds

In [ ]:
f, ax = plt.subplots(1, figsize=(7, 7))
sb_grace_lwe_thickness_ds.lwe_thickness.isel(time=6).plot(ax=ax)
sb_huc10_gdf.plot(ax=ax, edgecolor='white', facecolor='none');

## Discharge from USGS NWIS

Use `hydrodata.interactive_map`: for HUC10 subbasin boundaries, produce clickable site markers.

In [ ]:
m2 = hd.interactive_map(tuple(sb_bbox.total_bounds))

In [ ]:
for i, i_gdf in sb_huc10_gdf.iterrows():
    folium.GeoJson(
        i_gdf.geometry,
        style_function=lambda feature: {'color': 'black', 'fillOpacity': 0.0},
        tooltip=i_gdf['huc10']
    ).add_child(
        folium.Popup(f"<b>HUC10:</b><br> {i_gdf['huc10']}<br><b>watershed area:</b><br> {i_gdf['areasqkm']:.0f} km^2")
    ).add_to(m2)


m2.fit_bounds(m2.get_bounds())

In [ ]:
m2

<div class="alert alert-info" style="font-size:100%">

<b>HELP?</b> would be great to have some script that allowed us to subset guages to those within the above-selected date range!
    
Right now, I'm just clicking through the pins to find a site with an appropriate date range, and then entering that site number below (`nearestoutlet_sitecode = [manually selected site]`)
    
</div>

In [ ]:
nwis = hd.NWIS()

In [ ]:
nearestoutlet_sitecode = '14211010'

With `mmd=True`, `nwis.get_streamflow` converts cms (cubic meters / second) to mmd (mm/day) based on the contributing drainage area of the station.

In [ ]:
nwis_q_daily_ts = nwis.get_streamflow([nearestoutlet_sitecode], dates=(date_start, date_end), mmd=True)

In [ ]:
nwis_q_daily_ts.head()

Resample to daily and tweak it so it's consistent with the other watershed-scale time series. 

NOTE: how to input nearestoutlet_sitecode as a variable below (instead of column site name directly)??

In [ ]:
nwis_q_ts_df = nwis_q_daily_ts.resample('1M').sum()
nwis_q_ts_df.index = nwis_q_ts_df.index.month
nwis_q_ts_df.index.name = 'month'
nwis_q_ts_df.rename(columns={'USGS-14211010': 'disch'}, inplace=True)

nwis_q_ts_df

## Groundwater level from USGS

In [ ]:
sb_gw = pd.read_csv("data/ORgw.csv")

In [ ]:
sb_gw.rename(columns={'116034_72019_00003': 'gw'}, inplace=True)

In [ ]:
sb_gw.dtypes

In [ ]:
sb_gw.head()

In [ ]:
sb_gw['datetime'] =  pd.to_datetime(sb_gw['datetime'], format='%Y-%m-%d')
sb_gw.head()

In [ ]:
sb_gw=sb_gw.set_index(sb_gw.datetime)

In [ ]:
sb_gw=sb_gw.loc[slice(date_start,date_end)]
sb_gw

Aggregate to monthly (mean) and label units

In [ ]:
sb_gw_ts_df = sb_gw.resample('1M').mean()
sb_gw_ts_df.index = sb_gw_ts_df.index.month
sb_gw_ts_df.index.name = 'month'
sb_gw_ts_df = sb_gw_ts_df[['gw']]

sb_gw_ts_df.attrs['long_name'] = 'monthly mean water level - depth below surface'
sb_gw_ts_df.attrs['units'] = 'feet'

sb_gw_ts_df

## SWE from SNOTEL

In [ ]:
snow_stn = pd.read_csv("data/OR_DB.csv",dtype={'huc8':object})

In [ ]:
stn=snow_stn[snow_stn.huc8 == sb_huc08code]
stn

In [ ]:
import ulmo
wsdlurl = "http://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL"

def fetch(sitecode, variablecode, start_date, end_date):
    print(sitecode, variablecode, start_date, end_date)
    values_df = None
    try:
        #Request data from the server
        site_values = ulmo.cuahsi.wof.get_values(
            wsdlurl, 'SNOTEL:'+sitecode, variablecode, start=start_date, end=end_date
        )
        #Convert to a Pandas DataFrame   
        values_df = pd.DataFrame.from_dict(site_values['values'])
        #Parse the datetime values to Pandas Timestamp objects
        values_df['datetime'] = pd.to_datetime(values_df['datetime'])
        #Set the DataFrame index to the Timestamps
        values_df.set_index('datetime', inplace=True)
        #Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        #Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']
    except:
        print("Unable to fetch %s" % variablecode)
    
    return values_df


In [ ]:
stn_swe = pd.DataFrame(index=pd.date_range(start=date_start, end=date_end))
for sitecode in stn.code:
    try:
        swe = fetch(sitecode, variablecode='SNOTEL:WTEQ_D', start_date=date_start, end_date=date_end)
        stn_swe[sitecode] = swe.value
    except:
        print(sitecode, 'has no data')
        stn_swe[sitecode] = np.nan

In [ ]:
stn_swe

Aggregate to monthly (mean)

In [ ]:
stn_swe_ts_df = stn_swe.resample('1M').mean()
stn_swe_ts_df.index = stn_swe_ts_df.index.month
stn_swe_ts_df.index.name = 'month'

Subset to a single site (either one) and label units

In [ ]:
stn_swe_ts_df.rename(columns={'398_OR_SNTL': 'stn_swe'}, inplace=True) # just renaming one
stn_swe_ts_df = stn_swe_ts_df[['stn_swe']]

stn_swe_ts_df.attrs['long_name'] = 'monthly mean SNOTEL SWE'
stn_swe_ts_df.attrs['units'] = 'mm/month'
stn_swe_ts_df

----------------

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">Generate basin time series means using basin regions and mask (HUC10-level) </b></br>

### Create `regionmask` regions from a GeoDataFrame

<b>REGIONMASK LIMITATION:</b> <a href="https://github.com/mathause/regionmask/issues/151">regionmask is hard-wired to work only with data in lat-lon coordinates!</a>
</div>

In [ ]:
sb_huc10_regions = rm.from_geopandas(
    sb_huc10_gdf, names="huc10", name="huc10"
)

sb_huc10_regions

In [ ]:
sb_huc10_regions.plot(label="name");

### HUC10 watershed time series means

In [ ]:
def generate_huc10_ts(regions, ds, dsvarname, crsvar=None):
    """
    Use regionmask to create a gridded mask from the regions and the xarray dataset's
    grid resolution. Copy the variable attributes from the source DataSet to the 
    output DataArray.
    Returns a DataArray
    """
    mask3d = regions.mask_3D(ds, drop=False)

    # Latitude weighting for area-weighted averaging.
    # But it's probably unnecessary for this small geographical extent of the Upper Yakima
    weights = np.cos(np.deg2rad(ds.lat))
    regions_ts = ds[dsvarname].weighted(mask3d * weights).mean(dim=("lat", "lon"))
    # Assign dataarray name, and variable long name and units to region_ts, from ds[dsvarname]
    regions_ts.name = dsvarname
    regions_ts.attrs['long_name'] = ds[dsvarname].attrs['long_name']
    regions_ts.attrs['units'] = ds[dsvarname].attrs['units']
    
    # if present, drop crs variable b/c it's not needed anymore
    if crsvar in ds:
        regions_ts = regions_ts.drop_vars(crsvar)
    
    return regions_ts

#### Precipitation and SWE

In [ ]:
daymet_prcp_regions_ts_daily = generate_huc10_ts(sb_huc10_regions, sb_daymet_ds_prcp, 'prcp', crsvar='latitude_longitude')
daymet_swe_regions_ts_daily = generate_huc10_ts(sb_huc10_regions, sb_daymet_ds_swe, 'swe', crsvar='latitude_longitude')

Aggregate (sum) to monthly resolution

In [ ]:
daymet_prcp_regions_ts = daymet_prcp_regions_ts_daily.resample(time='1M').sum()
daymet_prcp_regions_ts.attrs['long_name'] = 'monthly total precipitation'
daymet_prcp_regions_ts.attrs['units'] = 'mm/month'

In [ ]:
daymet_swe_regions_ts = daymet_swe_regions_ts_daily.resample(time='1M').mean()
daymet_swe_regions_ts.attrs['long_name'] = 'monthly mean SWE'
daymet_swe_regions_ts.attrs['units'] = 'mm/month' # km/m2 == mm

#### Evapotranspiration

In [ ]:
ssebop_et_regions_ts = generate_huc10_ts(sb_huc10_regions, sb_ssebop_et_ds, 'et', crsvar='crs')

#### Water storage

In [ ]:
grace_lwe_thickness_regions_ts = generate_huc10_ts(sb_huc10_regions, sb_grace_lwe_thickness_ds, 'lwe_thickness')

Convert units to mm, and update units attribute accordingly.

In [ ]:
grace_lwe_thickness_regions_ts *= 10
grace_lwe_thickness_regions_ts.attrs['units'] = 'mm/month'

Plot spatial data summaries

In [ ]:
yrange = 600 # in mm

In [ ]:
f, ax = plt.subplots(ncols=4,figsize=(20, 4))
daymet_prcp_regions_ts.plot(ax=ax[0], hue="region")
ax[0].set_title('Precipitation')
ax[0].set_ylim(0, yrange)
daymet_swe_regions_ts.plot(ax=ax[1], hue="region")
ax[1].set_title('Snow Water Equivalent')
ax[1].set_ylim(0, yrange-500)
ssebop_et_regions_ts.plot(ax=ax[2], hue="region")
ax[2].set_title('Evapotranspiration')
ax[2].set_ylim(0, yrange)
grace_lwe_thickness_regions_ts.plot(ax=ax[3], hue="region")
ax[3].set_title('Water Storage')
ax[3].set_ylim(-200, -200 + yrange);

### Whole-watershed DataFrames

Generate sb-wide PPT, SWE, ET and LWE_thickness time series from HUC10-level values.

In [ ]:
sb_huc10_gdf['areaweight'] = sb_huc10_gdf.areasqkm.values / sb_huc10_gdf.areasqkm.sum()

In [ ]:
def generate_sb_ts(da, davarname):
    """
    Calculate whole-watershed time series based on area-weighted averaging of
    HUC10-level time series, converting from DataArray to DataFrame.
    Returns a cleaned and consistent DataFrame
    """
    areawt = xr.DataArray(
        sb_huc10_gdf.areasqkm.values / sb_huc10_gdf.areasqkm.sum(), 
        dims=('region'), 
        coords={'region': sb_huc10_gdf.index}
    )
    sb_ts = da.weighted(areawt).mean(dim=("region"))
    # Assign dataarray name
    sb_ts.name = davarname
    sb_ts_df = sb_ts.to_dataframe()
    sb_ts_df.index = sb_ts_df.index.month
    sb_ts_df.index.name = 'month'
    
    return sb_ts_df

#### Precipitation and SWE

In [ ]:
daymet_prcp_sb_ts_df = generate_sb_ts(daymet_prcp_regions_ts, 'prcp')
daymet_swe_sb_ts_df = generate_sb_ts(daymet_swe_regions_ts, 'swe')

#### Evapotranspiration

In [ ]:
ssebop_et_sb_ts_df = generate_sb_ts(ssebop_et_regions_ts, 'et')

#### Water storage

In [ ]:
grace_lwe_thickness_sb_tsdf = generate_sb_ts(grace_lwe_thickness_regions_ts, 'lwe_thickness')

add in well level, snotel, discharge

----------------

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">All-variable data frame </b></br>

HUC8 basin and monthly time resolution for precip, ET, discharge, SWE (Daymet), SWE (**single SNOTEL site in basin**), GW level (**single USGS site in basin**)

In [ ]:
monthly_df = pd.concat([daymet_prcp_sb_ts_df, ssebop_et_sb_ts_df, nwis_q_ts_df, daymet_swe_sb_ts_df, stn_swe_ts_df, sb_gw_ts_df], 
                              axis='columns')
monthly_df